In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import time
from helpers import permute_data, random_like

In [2]:
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor

In [3]:
DATA_PATH = Path("./data").resolve()

In [4]:
appl_price =  pd.read_parquet(f"{DATA_PATH}/AAPL.parquet").get("Close")

In [16]:
## regards to period, we can either split using vectorbt or just do the splitting ourselfs
## i.e data = data.loc[(data.index >= START_DATE) & (data.index < END_DATE)]
START_DATE = "<start here>"
END_DATE = "<end here>"

In [5]:
appl_price_truncated =  appl_price.iloc[:365]

In [7]:
windows = np.arange(5, 101, 5)

## Either the built-in, using pandas_ta or creating a custom one

In [10]:
#builtin
fast_ma, slow_ma = vbt.MA.run_combs(appl_price_truncated, window=windows, r=2, short_names=['fast', 'slow'])

In [13]:
#using indicator factory, these are more consistent for run_combs
x, y = vbt.pandas_ta("SMA").run_combs(appl_price_truncated, windows,short_names=['fast', 'slow']) 

SMA = vbt.IndicatorFactory.from_pandas_ta("SMA")
fast_sma, slow_sma = SMA.run_combs(appl_price_truncated, windows, short_names=['fast', 'slow'])

SMAInd = vbt.IndicatorFactory(input_names=['price'], param_names=['window'],
                              output_names=['ma']).from_apply_func(vbt.nb.rolling_mean_nb)
SMA_fast, SMA_slow = SMAInd.run_combs(appl_price_truncated, windows, short_names=['fast', 'slow'])

In [28]:
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

random_entries = entries.apply(random_like)
permuted_entries = entries.apply(permute_data)

In [29]:
pf_kwargs = dict(size=np.inf, fees=0.001, freq='1D')

In [30]:
pf = vbt.Portfolio.from_signals(appl_price_truncated, entries, exits, **pf_kwargs)
random_pf = vbt.Portfolio.from_signals(appl_price_truncated, random_entries, exits, **pf_kwargs)
permuted_pf = vbt.Portfolio.from_signals(appl_price_truncated, permuted_entries, exits, **pf_kwargs)

In [31]:
expectancy  = pf.trades.expectancy()
random_expectancy  = random_pf.trades.expectancy()
permute_expectancy = permuted_pf.trades.expectancy()

In [32]:
mean_expectancy = expectancy.mean()
mean_random_expectancy = random_expectancy.mean()
mean_permuted_expectancy = permute_expectancy.mean()

In [33]:
mean_expectancy, mean_permuted_expectancy, mean_random_expectancy

(-10.34032745916665, -9.621661090819039, -21.94558006648226)

## now we implement it so we can use multiprocessing

In [114]:
from abc import abstractmethod, ABCMeta

class Strategy(metaclass=ABCMeta):
    @abstractmethod
    def get_entries(self):
        """
        returns entries signals
        """
        pass   
    
    @abstractmethod
    def get_exits(self):
        """
        returns exit signals
        """
        pass 

In [200]:
class Backtest:
    def __init__(self, ticker:str, strategy:Strategy, fees:float = 0.001, size:float = np.inf, freq:str = "1D"):
        self.ticker=ticker
        self.strategy = strategy
        self.fees = fees
        self.size = size
        self.freq=freq
    
    def __repr__(self):
        return f"<Bactest {str(self.strategy)}>"
    
    def _get_portfolio(self, entries, exits):
        return vbt.Portfolio.from_signals(close=self.strategy.data,
                                          entries=entries, 
                                          exits=exits, 
                                          fees=self.fees,size = self.size, 
                                          freq=self.freq)
    
    def run(self, **kwargs):
        entries = self.strategy.get_entries()
        exits = self.strategy.get_exits()
        
        random_entries = entries.apply(random_like)
        permuted_entries = entries.apply(permute_data)
        
        portfolio = self._get_portfolio(entries, exits, **kwargs)
        random_porfolio = self._get_portfolio(random_entries, exits, **kwargs)
        permuted_portfolio = self._get_portfolio(permuted_entries, exits,**kwargs)
        
        output = {
            self.ticker : dict(
                mean_expectancy = portfolio.trades.expectancy().mean(),
                max_expectancy = portfolio.trades.expectancy().max(),
                mean_random_expectancy= random_porfolio.trades.expectancy().mean(),
                mean_permuted_expectancy = permuted_portfolio.trades.expectancy().mean()
            )
        }
        return output      

In [155]:
class SMAStrategy(Strategy):
    def __init__(self, data):
        self.data = data
        self.windows=  np.arange(10, 100, 5)
        self.indicator = vbt.IndicatorFactory.from_pandas_ta("SMA")
        self.fast_sma, self.slow_sma = self.indicator.run_combs(self.data, self.windows, short_names=['fast', 'slow'])
    
    def get_entries(self):
        return self.fast_sma.sma_crossed_above(self.slow_sma)
    
    def get_exits(self):
        return self.fast_sma.sma_crossed_below(self.slow_sma)   

In [209]:
from concurrent.futures import ProcessPoolExecutor
DATA_PATH = Path("./data").resolve() 

def run_backtest(ticker:str):
    print(f"backtesting on {ticker} \n")
    data = pd.read_parquet(f"{DATA_PATH/ticker}.parquet").get("Close")
    strategy = SMAStrategy(data)
    backtest = Backtest(ticker, strategy)
    result = backtest.run()
    return pd.DataFrame.from_dict(result, orient="index")   

In [211]:
with ProcessPoolExecutor() as executor:
    result = executor.map(run_backtest, tickers)
    
output = pd.concat(list(result))

backtesting on AAPL 
backtesting on AAL 
backtesting on AAP 
backtesting on AA 




backtesting on AAU 



In [214]:
output.to_csv("test_sma.csv")

PosixPath('/mnt/c/Users/Calculus/Desktop/Library/Courses/MscFE/10. Capstone/Code/data')

In [221]:
tickers = [file for file in DATA_PATH.iterdir()]

In [223]:
def delete_empty_file(file):
    if (len(pd.read_parquet(file)) == 0):
        print(f"deleting {file}")
        file.unlink()

In [ ]:
DATA_PATH

In [ ]:
with ProcessPoolExecutor() as executor:
    executor.map(delete_empty_file, tickers)